## STATUS UPDATES

## PART II

### Specific Aim
My capstone project involves examining public proceedings and debates of the U.S. House of Representatives and Senate during the Obama presidency through natural language processing. I am interested in comparing legislators' remarks during Obama's first term and second term and determining (1) whether there is a difference in the kind/sentiment of words spoken and (2) if it is possible to predict the passage of proposed legislation based on such a difference.

Questions to Consider
1. Do Senators speak differently than Representatives?
2. Do Republicans speak differently than Democrats?
3. Does a party's minority/majority status affect the language its party members use?
4. Does the quantity/type/sentiment of words used depend on seniority? gender? something else?

### Prosposed Methods and Models
After building my datasets of extracted words and legislative information, I plan to employ unsupervised and supervised learning techniques such as clustering to identify common topics/themes among spoken remarks and some kind of tree-based classifier to predict a bill's passage/failure. My anticipation is that the clusters will represent both generalized policy areas (defense, fiscal, economic, and social) and more specific, contentious policy concerns (federal budget, national debt, defunding Planned Parenthood, the Affordable Care Act). In spite of my fondness for hierarchical aglomerative clustering, I will likely end up using k-means and perhaps give DBSCAN a shot as well. The latter is iffy because it requires so much tuning and may take up too much time. 

This immersive has convinced me of the power of tree-based methods, especially when those methods are random forest, extra trees, and boosting. My plan is to fit a baseline model of each to the data and subsequently optimize it using grid search to find the best parameters. The performance metrics upon which I plan to focus are the accuracy score, and cross validation score, precision and recall. 

### Risks and Assumptions
One risk of this analysis is that the features might be all wrong: it might turn out that language has no bearing on a bill becoming a law or its significance might pale in comparison to that of other factors such as yay/nay votes as a consequence of partisanship and logrolling (trading votes). Although I want to restrict my analysis to debates about and remarks regarding specific pieces of legislation, it is possible that too few or too many observations were dropped, leaving me with a bad dataset. A related concern is rows with the title of ADDITIONAL STATEMENTS. Some legislators use these to recognize specific individuals, whereas others take this time to comment on a bill. It is going to take a lot of effort to separate the wheat from the chaff, so I'm going to keep all those observations and hope that substantive remarks will drown out the irrelevant ones. Another risk is the inclusion of filibuster speeches. While Ted Cruz reading Green Eggs and Ham on the Senate floor for his daughter might make for good television and a waste of everyone else's time, it adds noise to my model. 

### Data Cleaning/Munging
Unfortunately, I'm still stuck in this phase of my capstone. It took me a while to identify the best method for webscraping the Congressional Record (you would think that government would make this data easily accessible using an API), and the compilation of my dataset has yet to be completed. Years 2012 - 2017 have been scraped, converted to a DataFrame, then a CSV, and back to a DataFrame. Each of the columns has been changed into its proper type, preliminary cleaning has been performed on the text column, and I am currently in the process of dropping irrrelevant observations, which are composed of procedural/ceremonial actions such as reciting the Pledge of Allegiance, appointing the President Pro Tempore, and beginning each session with a prayer (seriously). Once the rows have been winnowed, features describing the word count and sentiment of each word will be generated. In terms of additional datasets, I plan to use ProPublica's Congress API to extract data on legislation and members of Congress.

### Exploratory Data Analysis and Visualizations
Forthcoming! 

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline

import json

import re

In [2]:
# years = ['12', '13', '14', '15', '16', '17']
# file_path_template = ['concord/congress_gov/logs_and_output/output_', '.json']

# for year in years:
#     json_file = file_path_template[0] + year + file_path_template[1]
#     with open(json_file, 'r') as output:
#         json_data = [json.loads(line) for line in output]

In [3]:
# json_data

In [4]:
# big_df = pd.DataFrame(columns = ['congress', 'date', 'end_page', 'issue', 'session', 'start_page', 'text', 'title', 'url', 'volume'])

# for i in json_data:
#     dict_x = pd.DataFrame(i, index = range(len(i)))
#     big_df = pd.concat([big_df, dict_x], axis = 0)

In [5]:
# cr_12_17 = big_df.reset_index(drop = True)

In [6]:
# DO NOT RELOAD!
# cr_12_17.to_csv('XX/Users/sidraahmad/DSI-NYC-4/capstone/datasets/cr_12_17_notclean_utf8.csv', index = False, encoding = 'utf-8')

In [7]:
cr_12_17 = pd.read_csv('/Users/sidraahmad/DSI-NYC-4/capstone/datasets/cr_12_17_notclean_utf8.csv')
cr_12_17.head()

,congress,date,end_page,issue,session,start_page,text,title,url,volume
0,112,01/03/2012,H1,1,2,H1,\n \n\n[Page H1]\nFrom the Congressiona...,ADJOURNMENT,https://www.congress.gov/congressional-record/...,158
1,112,01/03/2012,H1,1,2,H1,\n \n\n[Page H1]\nFrom the Congressiona...,ADJOURNMENT,https://www.congress.gov/congressional-record/...,158
2,112,01/03/2012,H1,1,2,H1,\n \n\n[Page H1]\nFrom the Congressiona...,ADJOURNMENT,https://www.congress.gov/congressional-record/...,158
3,112,01/03/2012,H1,1,2,H1,\n \n\n[Page H1]\nFrom the Congressiona...,ADJOURNMENT,https://www.congress.gov/congressional-record/...,158
4,112,01/03/2012,H1,1,2,H1,\n \n\n[Page H1]\nFrom the Congressiona...,ADJOURNMENT,https://www.congress.gov/congressional-record/...,158


In [8]:
cr_12_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156900 entries, 0 to 156899
Data columns (total 10 columns):
congress      156900 non-null int64
date          156900 non-null object
end_page      156900 non-null object
issue         156900 non-null int64
session       156900 non-null int64
start_page    156900 non-null object
text          156900 non-null object
title         156900 non-null object
url           156900 non-null object
volume        156900 non-null int64
dtypes: int64(4), object(6)
memory usage: 12.0+ MB


In [9]:
cr_12_17['date'] = cr_12_17['date'].apply(lambda x: pd.to_datetime(x))

In [10]:
cr_12_17['text'][0]

"\n        \n\n[Page H1]\nFrom the Congressional Record Online through the Government Publishing Office [www.gpo.gov]\n\n\n\n\n                              ADJOURNMENT\n\n  The SPEAKER pro tempore. Pursuant to section 4(c) of House Resolution \n493, the House stands adjourned until 10 a.m. on Friday, January 6, \n2012.\n  Accordingly (at 12 o'clock and 2 minutes p.m.), the House adjourned \nuntil Friday, January 6, 2012, at 10 a.m.\n\n\n\n    "

In [11]:
cr_12_17['text'] = [x.replace('\n', '') for x in cr_12_17['text']]
cr_12_17['text'] = [x.strip() for x in cr_12_17['text']]
cr_12_17['text'] = cr_12_17['text'].apply(lambda x: re.sub(r'\[(.*)\]\s*\W', '', x))

In [15]:
drop_vals = ['PLEDGE OF ALLEGIANCE', 'RECESS', 'ANNOUNCEMENT BY THE SPEAKER PRO TEMPORE', 'THE JOURNAL', 
                          'MORNING BUSINESS', 'AFTER RECESS', 'ADJOURNMENT', 'PRAYER', 
                          'APPOINTMENT OF ACTING PRESIDENT PRO TEMPORE', 'EXECUTIVE COMMUNICATIONS, ETC.', 
                          'ADDITIONAL SPONSORS', 'PUBLIC BILLS AND RESOLUTIONS', 'Constitutional Authority Statement', 
                          'prayer', 'ADDITIONAL COSPONSORS', 'RECOGNITION OF THE MAJORITY LEADER', 
                          'AMENDMENTS SUBMITTED AND PROPOSED', 'TEXT OF AMENDMENTS', 'SUBMITTED RESOLUTIONS', 
                          'EXECUTIVE AND OTHER COMMUNICATIONS', 'AUTHORITY FOR COMMITTEES TO MEET', 
                          'LEAVE OF ABSENCE']

for i in drop_vals:
    cr_12_17 = cr_12_17[cr_12_17.title != i]

In [ ]:
drop_more_vals = ['', '']

for i in drop_more_vals:
    cr_12_17 = cr_12_17[cr_12_17.title != i]

In [16]:
cr_12_17['title'].value_counts()

Senate                                                                                                                          1710
House of Representatives                                                                                                        1470
INTRODUCTION OF BILLS AND JOINT RESOLUTIONS                                                                                     1180
ADDITIONAL STATEMENTS                                                                                                           1150
PROGRAM                                                                                                                         1130
REPORTS OF COMMITTEES ON PUBLIC BILLS AND RESOLUTIONS                                                                           1060
SUBMISSION OF CONCURRENT AND SENATE RESOLUTIONS                                                                                  980
STATEMENTS ON INTRODUCED BILLS AND JOINT RESOLUTIONS                 

In [17]:
cr_12_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126470 entries, 10 to 156899
Data columns (total 10 columns):
congress      126470 non-null int64
date          126470 non-null datetime64[ns]
end_page      126470 non-null object
issue         126470 non-null int64
session       126470 non-null int64
start_page    126470 non-null object
text          126470 non-null object
title         126470 non-null object
url           126470 non-null object
volume        126470 non-null int64
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 10.6+ MB


In [ ]:
Questionable Columns = ['INTRODUCTION OF BILLS AND JOINT RESOLUTIONS', 'ADDITIONAL STATEMENTS', 
                        'REPORTS OF COMMITTEES ON PUBLIC BILLS AND RESOLUTIONS', 'AMENDMENTS SUBMITTED AND PROPOSED', 
                        'SUBMISSION OF CONCURRENT AND SENATE RESOLUTIONS', '']

In [11]:
cr_12_17['title'].value_counts()

PLEDGE OF ALLEGIANCE                                                                                                            2710
RECESS                                                                                                                          1890
Senate                                                                                                                          1710
ANNOUNCEMENT BY THE SPEAKER PRO TEMPORE                                                                                         1710
THE JOURNAL                                                                                                                     1690
MORNING BUSINESS                                                                                                                1610
AFTER RECESS                                                                                                                    1540
ADJOURNMENT                                                          

In [12]:
cr_12_17['title'].nunique()

8480

In [23]:
cr_12_17['txtlen'] = cr_12_17['text'].apply(len)
cr_12_17 = cr_12_17.sort_values(by = 'txtlen', ascending = False)

cr_12_17['txtlen']

150178    4124416
150173    4124416
150177    4124416
150171    4124416
150172    4124416
150170    4124416
150174    4124416
150175    4124416
150176    4124416
150179    4124416
83592     2048196
83597     2048196
83596     2048196
83598     2048196
83591     2048196
83590     2048196
83593     2048196
83594     2048196
83595     2048196
83599     2048196
59853     1860746
59852     1860746
59851     1860746
59850     1860746
59856     1860746
59854     1860746
59855     1860746
59857     1860746
59858     1860746
59859     1860746
           ...   
76297         229
76292         229
76294         229
76295         229
76296         229
76291         229
76298         229
76290         229
76293         229
76299         229
142269        197
142262        197
142263        197
142264        197
142265        197
142266        197
142267        197
142268        197
142260        197
142261        197
106525        196
106522        196
106523        196
106520        196
106521    